CASO COORDINADOR(A) RGM:

Puntos 1, 2 y 3: 
1. Generar un DataFrame con los datos de los cuatro ficheros
2. Filtrar las columnas del DataFrame para quedarse con las columnas ESTACION, MAGNITUD, AÑO, MES y las correspondientes a los días D01, D02, etc.
(Reto separar las columnas en el df_total)
3. Reestructurar el DataFrame para que los valores de los contaminantes de las columnas de los días aparezcan en una única columna (Reto no conocía la función melt)


In [2]:
#Importar librería
import pandas as pd

#Leer ficheros teniendo en cuenta que está separado por ;

df_2016 = pd.read_csv('C:/Users/J/Documents/Prueba coordinador RGM/emisiones-2016.csv', sep=';')
df_2017 = pd.read_csv('C:/Users/J/Documents/Prueba coordinador RGM/emisiones-2017.csv',sep=';')
df_2018 = pd.read_csv('C:/Users/J/Documents/Prueba coordinador RGM/emisiones-2018.csv',sep=';')
df_2019 = pd.read_csv('C:/Users/J/Documents/Prueba coordinador RGM/emisiones-2019.csv',sep=';')

#juntar ficheros

df_total = pd.concat([df_2016, df_2017, df_2018, df_2019], ignore_index=True)

#df campos solicitados

columnas_df_campos = ['ESTACION', 'MAGNITUD', 'ANO', 'MES', 'D01', 'D02', 'D03', 'D04', 'D05', 'D06', 'D07', 'D08', 'D09', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'D16', 'D17', 'D18', 'D19', 'D20', 'D21', 'D22', 'D23', 'D24', 'D25', 'D26', 'D27', 'D28', 'D29', 'D30', 'D31']

df_filtrado = df_total[columnas_df_campos]

#Restructurar el Data frame para que los valores de los contaminantes de las columnas de los días aparezcan en una única columna (Pude haber usado un ciclo for)

df_reestructurado = df_filtrado.melt(id_vars=['ESTACION', 'MAGNITUD', 'ANO', 'MES'],
                                     value_vars=['D01', 'D02', 'D03', 'D04', 'D05', 'D06', 'D07', 'D08', 'D09', 'D10',
                                                 'D11', 'D12', 'D13', 'D14', 'D15', 'D16', 'D17', 'D18', 'D19', 'D20',
                                                 'D21', 'D22', 'D23', 'D24', 'D25', 'D26', 'D27', 'D28', 'D29', 'D30', 'D31'],
                                     var_name='DIA',
                                     value_name='VALOR_CONTAMINANTE')
#Cambiar 'DIA' que está codificado por un número

df_reestructurado['DIA'] = df_reestructurado['DIA'].str.extract('(\d+)').astype(int)



#Pasar AÑO y MES a enteros
df_reestructurado['ANO'] = df_reestructurado['ANO'].astype(int)
df_reestructurado['MES'] = df_reestructurado['MES'].astype(int)


df_reestructurado.head()


<>:31: SyntaxWarning: invalid escape sequence '\d'
<>:31: SyntaxWarning: invalid escape sequence '\d'
C:\Users\J\AppData\Local\Temp\ipykernel_26020\127801928.py:31: SyntaxWarning: invalid escape sequence '\d'
  df_reestructurado['DIA'] = df_reestructurado['DIA'].str.extract('(\d+)').astype(int)


,ESTACION,MAGNITUD,ANO,MES,DIA,VALOR_CONTAMINANTE
0,4,1,2016,1,1,8.0
1,4,1,2016,2,1,12.0
2,4,1,2016,3,1,11.0
3,4,1,2016,4,1,8.0
4,4,1,2016,5,1,7.0


4. Añadir una columna con la fecha a partir de la concatenación del año, el mes y el día (usar el módulo `datetime`) (Reto, encontrar fórmula que realmente omita las fechas inválidas y las pase a NaT)

In [3]:
from datetime import datetime

# Crear la columna 'FECHA' manejando fechas inválidas
def safe_datetime(year, month, day):
    try:
        return datetime(year, month, day)
    except ValueError:
        return None  

# Aplicar la función a cada fila del DataFrame
df_reestructurado['FECHA'] = df_reestructurado.apply(lambda row: safe_datetime(int(row['ANO']), int(row['MES']), int(row['DIA'])), axis=1)

print(df_reestructurado)


        ESTACION  MAGNITUD   ANO  MES  DIA  VALOR_CONTAMINANTE      FECHA
0              4         1  2016    1    1                 8.0 2016-01-01
1              4         1  2016    2    1                12.0 2016-02-01
2              4         1  2016    3    1                11.0 2016-03-01
3              4         1  2016    4    1                 8.0 2016-04-01
4              4         1  2016    5    1                 7.0 2016-05-01
...          ...       ...   ...  ...  ...                 ...        ...
225241        60        14  2019    8   31                98.0 2019-08-31
225242        60        14  2019    9   31                 0.0        NaT
225243        60        14  2019   10   31                47.0 2019-10-31
225244        60        14  2019   11   31                 0.0        NaT
225245        60        14  2019   12   31                 4.0 2019-12-31

[225246 rows x 7 columns]


In [15]:
df_reestructurado.to_excel('Base general prueba.xlsx',index=False)

5.1. Eliminar las filas con fechas no válidas (utilizar la función `isnat` del módulo `numpy`) 

In [5]:
import numpy as np

# Primero, combinamos las condiciones usando np.isnan() para NaN y np.isnat() para fechas no válidas
condicion_invalida = (
    np.isnat(df_reestructurado['FECHA']))


# Filtrar el DataFrame para eliminar las filas no válidas
df_valido = df_reestructurado[~condicion_invalida]

# Mostrar las primeras filas del DataFrame filtrado
print(df_valido)

        ESTACION  MAGNITUD   ANO  MES  DIA  VALOR_CONTAMINANTE      FECHA
0              4         1  2016    1    1                 8.0 2016-01-01
1              4         1  2016    2    1                12.0 2016-02-01
2              4         1  2016    3    1                11.0 2016-03-01
3              4         1  2016    4    1                 8.0 2016-04-01
4              4         1  2016    5    1                 7.0 2016-05-01
...          ...       ...   ...  ...  ...                 ...        ...
225238        60        14  2019    5   31                85.0 2019-05-31
225240        60        14  2019    7   31                92.0 2019-07-31
225241        60        14  2019    8   31                98.0 2019-08-31
225243        60        14  2019   10   31                47.0 2019-10-31
225245        60        14  2019   12   31                 4.0 2019-12-31

[221158 rows x 7 columns]


5.2. Ordenar el DataFrame por estaciones contaminantes y fecha

In [6]:
# Ordenar el DataFrame por 'ESTACION' y 'FECHA' en orden ascendente
df_valido = df_valido.sort_values(by=['ESTACION', 'FECHA'])

print(df_valido)


        ESTACION  MAGNITUD   ANO  MES  DIA  VALOR_CONTAMINANTE      FECHA
0              4         1  2016    1    1                 8.0 2016-01-01
12             4         6  2016    1    1                 0.4 2016-01-01
24             4         7  2016    1    1                28.0 2016-01-01
36             4         8  2016    1    1                40.0 2016-01-01
48             4        12  2016    1    1                83.0 2016-01-01
...          ...       ...   ...  ...  ...                 ...        ...
225197        60         7  2019   12   31                77.0 2019-12-31
225209        60         8  2019   12   31                83.0 2019-12-31
225221        60        10  2019   12   31                25.0 2019-12-31
225233        60        12  2019   12   31               202.0 2019-12-31
225245        60        14  2019   12   31                 4.0 2019-12-31

[221158 rows x 7 columns]


6. Mostrar por pantalla las estaciones y los contaminantes disponibles en el DataFrame. (El mayor reto es entender las variables, teniendo en cuenta el desarrollo del ejercicio tomé el campo "MAGNITUD" como el tipo de contaminante, para el momento de desarrollo de la prueba no pude confirmar mi hipótesis, sin embargo, la asumí ya que era el campo que más se aproximaba a la descripción de "Contaminante disponible" en caso de ser otro campo se puede hacer el cambio de campo en este y los siguientes puntos.)

In [7]:
# Agrupar por 'ESTACION' y obtener las magnitudes únicas en cada una,tomese MAGNITUD como tipo de contaminante
magnitudes_por_estacion = df_valido.groupby('ESTACION')['MAGNITUD'].unique().reset_index()

magnitudes_por_estacion.head(30)


,ESTACION,MAGNITUD
0,4,"[1, 6, 7, 8, 12]"
1,8,"[1, 6, 7, 8, 9, 10, 12, 14, 20, 30, 35, 42, 43..."
2,11,"[7, 8, 12, 20, 30, 35]"
3,16,"[6, 7, 8, 12, 14]"
4,17,"[1, 7, 8, 12, 14]"
5,18,"[1, 6, 7, 8, 10, 12, 14, 20, 30, 35]"
6,24,"[1, 6, 7, 8, 9, 10, 12, 14, 20, 30, 35, 42, 43..."
7,27,"[7, 8, 12, 14]"
8,35,"[1, 6, 7, 8, 12, 14]"
9,36,"[1, 6, 7, 8, 10, 12]"


7. Crear una función que reciba una estación, un contaminante y un rango de fechas y devuelva una serie con las emisiones del contaminante dado en la estación y rango de fechas dado.

In [8]:
import pandas as pd

def obtener_serie_tiempo(df, estacion, magnitud, fecha_inicio, fecha_fin):
   
    df_filtrado = df[(df['ESTACION'] == estacion) & (df['MAGNITUD'] == magnitud)]
    df_filtrado = df_filtrado[(df_filtrado['FECHA'] >= fecha_inicio) & (df_filtrado['FECHA'] <= fecha_fin)]
    serie_tiempo = df_filtrado.set_index('FECHA')['VALOR_CONTAMINANTE']
    
    return serie_tiempo

# Ejemplo
serie = obtener_serie_tiempo(df_valido, estacion=4, magnitud=1, fecha_inicio='2019-01-01', fecha_fin='2019-01-15')
print(serie)



FECHA
2019-01-01    18.0
2019-01-02    20.0
2019-01-03    18.0
2019-01-04    19.0
2019-01-05    18.0
2019-01-06    18.0
2019-01-07    21.0
2019-01-08    20.0
2019-01-09    18.0
2019-01-10    13.0
2019-01-11    16.0
2019-01-12    16.0
2019-01-13    16.0
2019-01-14    22.0
2019-01-15    21.0
Name: VALOR_CONTAMINANTE, dtype: float64


8. Mostrar un resumen descriptivo (mínimo, máximo, media, etc.) para cada contaminante.

In [9]:

# Agrupé por 'MAGNITUD' 
resumen_descriptivo = df_valido.groupby('MAGNITUD')['VALOR_CONTAMINANTE'].describe()

# Mostrar el resumen descriptivo
resumen_descriptivo.head(30)


,count,mean,std,min,25%,50%,75%,max
MAGNITUD,,,,,,,,
1,14610.0,7.428953,7.012504,0.00,4.00,7.00,10.00,610.00
6,14610.0,0.350233,0.215935,0.00,0.20,0.30,0.40,14.90
7,35064.0,20.446412,135.123509,0.00,4.00,9.00,23.00,24742.00
8,35064.0,37.677618,20.118050,0.00,22.00,35.00,50.00,148.00
9,8948.0,10.087729,10.643591,0.00,6.00,9.00,13.00,850.00
10,17897.0,18.772923,35.723619,0.00,10.00,16.00,24.00,4481.00
12,35064.0,67.959417,61.443940,0.00,29.00,48.00,84.00,1005.00
14,20454.0,49.941772,24.753120,0.00,31.00,52.00,69.00,336.00
20,8766.0,2.364944,4.236706,0.00,0.80,1.60,2.80,195.00


9. Mostrar un resumen descriptivo para cada contaminante por distritos. Se tomó distrito por estación, teniendo en cuenta puntos siguientes y al notar que en los fichetos provincia y municipio siempre tenían el mismo valor.

In [10]:
# Agrupé por 'ESTACION' y 'MAGNITUD', la descripción la saqué de 'VALOR_CONTAMINANTE'
resumen_descriptivo_estacion = df_valido.groupby(['ESTACION', 'MAGNITUD'])['VALOR_CONTAMINANTE'].describe()

resumen_descriptivo_estacion.head(30)

count       mean        std    min    25%    50%     75%  \
ESTACION MAGNITUD                                                              
4        1         1461.0   7.329911  16.379050   1.00   4.00   7.00    9.00   
         6         1461.0   0.411499   0.172902   0.10   0.30   0.40    0.50   
         7         1461.0  31.939767  37.667968   0.00   8.00  16.00   42.00   
         8         1461.0  44.398357  17.766063   0.00  31.00  43.00   55.00   
         12        1461.0  93.341547  72.436531   0.00  44.00  69.00  119.00   
8        1         1461.0   8.375770   4.364607   0.00   5.00   8.00   11.00   
         6         1461.0   0.362628   0.161711   0.00   0.30   0.30    0.40   
         7         1461.0  28.396988  29.909610   0.00  10.00  18.00   34.00   
         8         1461.0  55.792608  19.074047   0.00  43.00  54.00   67.00   
         9         1461.0  10.772074   6.147006   0.00   6.00  10.00   14.00   
         10        1461.0  20.474333  13.037940   0.00  12.00  18.00   26.00   
         12        1461.0  99.309377  61.952358   0.00  60.00  83.00  118.00   
         14        1461.0  43.512663  21.045434   0.00  27.00  44.00   60.00   
         20        1461.0   2.978097   2.369885   0.00   1.60   2.30    3.70   
         30        1461.0   0.753457   0.441863   0.00   0.50   0.60    0.90   
         35        1461.0   0.588501   0.830096   0.00   0.20   0.40    0.60   
         42        1461.0   1.577611   0.258008  -0.01   1.46   1.56    1.68   
         43        1461.0   1.463012   0.226152  -0.05   1.38   1.47    1.55   
         44        1461.0   0.115688   0.072236   0.00   0.07   0.11    0.14   
11       7         1461.0  21.675565  25.685535   1.00   7.00  13.00   25.00   
         8         1461.0  42.932238  19.280700   9.00  29.00  40.00   55.00   
         12        1461.0  76.201232  56.277571  11.00  40.00  60.00   92.00   
         20        1461.0   2.767625   5.628113   0.00   0.90   1.70    3.70   
         30        1461.0   0.736550   0.726574   0.00   0.20   0.40    1.00   
         35        1461.0   0.388980   0.620976   0.00   0.10   0.20    0.40   
16       6         1461.0   0.314237   0.141140   0.10   0.20   0.30    0.40   
         7         1461.0  15.732375  22.400837   1.00   3.00   7.00   18.00   
         8         1461.0  37.699521  17.539448   6.00  25.00  35.00   49.00   
         12        1461.0  61.802875  49.655313   9.00  30.00  46.00   76.00   
         14        1461.0  45.262149  23.059372   0.00  28.00  45.00   63.00   

                      max  
ESTACION MAGNITUD          
4        1         610.00  
         6           1.30  
         7         239.00  
         8         105.00  
         12        467.00  
8        1          24.00  
         6           1.30  
         7         230.00  
         8         148.00  
         9          84.00  
         10        233.00  
         12        501.00  
         14        103.00  
         20         35.30  
         30          3.60  
         35         13.90  
         42          3.09  
         43          2.77  
         44          1.31  
11       7         194.00  
         8         119.00  
         12        385.00  
         20        195.00  
         30          4.20  
         35         17.80  
16       6           1.20  
         7         179.00  
         8         110.00  
         12        384.00  
         14        108.00

10. Crear una función que reciba una estación y un contaminante y devuelva un resumen descriptivo de las emisiones del contaminante indicado en la estación indicada.

In [11]:

def resumen_emisiones(df, estacion, contaminante):

    # Filtrar el DataFrame por la estación y el contaminante
    df_filtrado = df[(df['ESTACION'] == estacion) & (df['MAGNITUD'] == contaminante)]
    
    # Verificar si el DataFrame filtrado tiene datos
    if df_filtrado.empty:
        return pd.DataFrame() 
    
    resumen = df_filtrado['VALOR_CONTAMINANTE'].describe()
    
    return resumen

# Ejemplo
resumen = resumen_emisiones(df_valido, estacion=4, contaminante=1)

print(resumen)


count    1461.000000
mean        7.329911
std        16.379050
min         1.000000
25%         4.000000
50%         7.000000
75%         9.000000
max       610.000000
Name: VALOR_CONTAMINANTE, dtype: float64


11. Crear una función que devuelva las emisiones medias mensuales de un contaminante y un año dados para todas las estaciones.

In [12]:
def emisiones_mensuales_medias(df, contaminante, año):
    
    df_valido = df[(df['MAGNITUD'] == contaminante) & (df['FECHA'].dt.year == año)]
   
    if df_valido.empty:
        return pd.DataFrame() 

    df_valido['MES'] = df_valido['FECHA'].dt.month
    emisiones_mensuales = df_valido.groupby('MES')['VALOR_CONTAMINANTE'].mean().reset_index()
    
    # Cambio nombre de columnas
    emisiones_mensuales.columns = ['Mes', 'Emision_Media']
    
    return emisiones_mensuales

# Ejemplo
emisiones_mensuales = emisiones_mensuales_medias(df_valido, contaminante=1, año=2017)

print(emisiones_mensuales)


    Mes  Emision_Media
0     1      10.341935
1     2       8.796429
2     3       8.209677
3     4       5.483333
4     5       5.106452
5     6       5.513333
6     7       5.361290
7     8       6.187097
8     9       6.933333
9    10       5.058065
10   11       8.060000
11   12       7.729032


C:\Users\J\AppData\Local\Temp\ipykernel_26020\1517138098.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_valido['MES'] = df_valido['FECHA'].dt.month


12. Crear una función que reciba una estación de medición y devuelva un DataFrame con las medias mensuales de los distintos tipos de contaminantes.

In [13]:

def medias_mensuales_por_estacion(df, estacion):

    df_valido = df[df['ESTACION'] == estacion]

    if df_valido.empty:
        return pd.DataFrame()
    
    # En este caso extraje el mes y el año de la columna 'FECHA', pude haber utilizado losc
    df_valido['MES'] = df_valido['FECHA'].dt.month
    df_valido['AÑO'] = df_valido['FECHA'].dt.year
  
    medias_mensuales = df_valido.groupby(['AÑO', 'MES', 'MAGNITUD'])['VALOR_CONTAMINANTE'].mean().unstack()
    
    # Renombrado de columnas
    medias_mensuales.columns.name = None 
    medias_mensuales.reset_index(inplace=True)
    
    return medias_mensuales

# Ejemplo
medias_mensuales = medias_mensuales_por_estacion(df_valido, estacion=4)

medias_mensuales.head(60)


C:\Users\J\AppData\Local\Temp\ipykernel_26020\859283709.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_valido['MES'] = df_valido['FECHA'].dt.month
C:\Users\J\AppData\Local\Temp\ipykernel_26020\859283709.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_valido['AÑO'] = df_valido['FECHA'].dt.year


,AÑO,MES,1,6,7,8,12
0,2016,1,8.354839,0.432258,45.516129,42.935484,112.677419
1,2016,2,8.551724,0.368966,35.586207,37.241379,91.827586
2,2016,3,8.612903,0.370968,24.483871,40.129032,77.806452
3,2016,4,8.066667,0.330000,20.233333,37.733333,68.533333
4,2016,5,7.354839,0.300000,17.193548,36.774194,63.096774
5,2016,6,7.266667,0.273333,17.800000,40.300000,67.633333
6,2016,7,7.032258,0.248387,11.225806,41.709677,59.032258
7,2016,8,7.000000,0.251613,8.225806,37.387097,50.064516
8,2016,9,8.100000,0.346667,29.866667,57.400000,103.166667
9,2016,10,7.387097,0.490323,61.645161,60.387097,154.870968
